In [1]:
import pandas as pd
import numpy as np

Zu den Daten:
* von OAG.com auf Anfrage erhalten (sind zu erwähnen)
* Erfasst sind alle Flüge, zu denen Sitze angeboten wurden, die ZRH als Abflughafen haben.
* Erfasst sind Seats (= angebotene Sitze) und Frequency (=Flüge pro Jahr)

Zu diesem Notebook:
* Im Notebook OAG sind die Daten per Flughafen aufbereitet. Weil die BfS-Daten in den anderen Notebooks aber per Ort aufbereitet sind, schauen wir uns den OAG-Datensatz auch per Ort an.

In [2]:
path = "ZRH Data.xlsx"

In [3]:
df = pd.read_excel(path, skiprows=24, usecols='A:E') #In den ersten 24 Reihen stehen Angaben zum Datensatz, die wir nicht in der Tabelle brauchen; In den Spalten nach E ist eine Pivot-Tabelle enthalten (deren Ergebnisse stellen wir eh selber nach)

In [4]:
df.drop([1594,1595,1596], inplace=True) #In den letzten drei Zeilen sind Fusszeilen enthalten

In [5]:
df['Seats (Total)'] = df['Seats (Total)'].astype(int)
df['Frequency'] = df['Frequency'].astype(int)
df['Time series'] = df['Time series'].astype(int)

In [6]:
# #Wir gehen die Flughäfen einzeln durch und erfassen den Ort.
# airports = df['Arr Airport Name'].unique().tolist()
# orte = [] #In dieser Liste erfassen wir Dictionaries (Flughafen: Ort)
# for airport in airports:
#     ort = input("In welchem Ort steht der Flughafen " + str(airport) + "? ")
#     mini_dict = {'Flughafen': airport,
#                  'Ort': ort}
#     orte.append(mini_dict)
# df_orte = pd.DataFrame(orte) 
# df_orte.to_csv("Orte.csv") #Resultat speichern für weitere Verwendung.

In [7]:
#Die obige Zuteilung machen wir nicht jedes Mal neu. Wir können sie aber bei Bedarf aus dem CSV-File wiederherstellen.
df_orte = pd.read_csv("Orte.csv")
df_orte.drop('Unnamed: 0', axis=1, inplace=True)

In [8]:
#Wir mergen das ursprüngliche Dataframe mit dem Dataframe mit den Orten
df = pd.merge(df, df_orte, left_on="Arr Airport Name", right_on="Flughafen")
df.drop("Arr Airport Name", axis=1, inplace=True) #Die Flughafenangabe hätten wir sonst doppelt

In [9]:
#Nach Destinationen zusammenfassen
df_destinationen = df.groupby(['Ort','Time series']).sum().reset_index()

In [10]:
#Wir pivotieren das Dataframe und trennen dabei gleich Flüge und Sitze
df_seats = df_destinationen.pivot(index='Time series', columns=['Ort'], values='Seats (Total)')
df_flights = df_destinationen.pivot(index='Time series', columns=['Ort'], values='Frequency')
df_seats = df_seats.fillna(0) #Nan durch 0 ersetzen
df_flights = df_flights.fillna(0) #Nan durch 0 ersetzen

In [11]:
#Die Sitze pro Destination und Jahr können im Artikel als durchsuchbare Tabelle angehängt werden.
df_seats.to_csv('Sitze pro Destination.csv')

In [12]:
#Zum Vergleich der Bedeutung eines Zielflughafens sind relative Angaben sinnvoller.
df_seats_rel = df_seats.div(df_seats.sum(axis=1), axis=0) * 100
df_flights_rel = df_flights.div(df_flights.sum(axis=1), axis=0) * 100

In [36]:
#Die drei bedeutendsten Orte pro Jahr (Sitze) in ein Dataframe dftop schreiben.
df_seats_top = pd.DataFrame(df_seats_rel.columns.values[np.argsort(-df_seats_rel.values, axis=1)[:, :3]],
                  index=df_seats_rel.index,
                  columns = ['1st Max','2nd Max','3rd Max']).reset_index()
df_seats_top
df_seats_top.to_excel('Podest.xlsx') #Daten für Infografiker

,Time series,1st Max,2nd Max,3rd Max
0,2017,London,Berlin,Wien
1,2018,London,Berlin,Wien
2,2019,London,Berlin,Wien
3,2020,London,Berlin,Amsterdam
4,2021,London,Istanbul,Palma De Mallorca


In [14]:
#Die drei bedeutendsten Zielflughäfen pro Jahr (Flüge) in ein Dataframe dftop schreiben.
df_flights_top = pd.DataFrame(df_flights_rel.columns.values[np.argsort(-df_flights_rel.values, axis=1)[:, :3]],
                  index=df_flights_rel.index,
                  columns = ['1st Max','2nd Max','3rd Max']).reset_index()
df_flights_top

,Time series,1st Max,2nd Max,3rd Max
0,2017,London,Berlin,Amsterdam
1,2018,London,Berlin,Wien
2,2019,London,Berlin,Wien
3,2020,London,Amsterdam,Berlin
4,2021,London,Frankfurt,Amsterdam


In [15]:
#Veränderungen zum Vorjahr
df_seats_rel = df_seats_rel.fillna(0)
df_flights_rel = df_flights_rel.fillna(0)

In [16]:
#Die prozentuale Veränderung der Bedeutung pro angeflogenem Flughafen berechnen wir von Jahr zu Jahr (Unterschied = Prozentpunkte)
df_seats_rel_vorjahr = df_seats_rel.diff(1)
df_flights_rel_vorjahr = df_flights_rel.diff(1)

In [17]:
#Veränderungen zu 2019
#Wir erstellen je zwei Dataframes: Eines mit den 2019er-Zahlen, eines mit denen danach
df_seats_2019 = df_seats.iloc[2,:].copy()
df_seats_pandemie = df_seats.iloc[3:5,:].copy()
df_seats_rel_2019 = df_seats_rel.iloc[2,:].copy()
df_seats_rel_pandemie = df_seats_rel.iloc[3:5,:].copy()
df_flights_2019 = df_flights.iloc[2,:].copy()
df_flights_pandemie = df_flights.iloc[3:5,:].copy()
df_flights_rel_2019 = df_flights_rel.iloc[2,:].copy()
df_flights_rel_pandemie = df_flights_rel.iloc[3:5,:].copy()

In [18]:
df_seats_unterschied = df_seats_pandemie - df_seats_2019
df_seats_rel_unterschied = df_seats_rel_pandemie - df_seats_rel_2019
df_flights_unterschied = df_flights_pandemie - df_flights_2019
df_flights_rel_unterschied = df_flights_rel_pandemie - df_flights_rel_2019

In [19]:
#Grösste Rückgänge in absoluten Zahlen pro Jahr (Seats)
loser_names = df_seats_unterschied.idxmin(axis=1)
loser_values = df_seats_unterschied.min(axis=1)
loser = pd.DataFrame(loser_names)
loser['Values'] = loser_values
loser

,0,Values
Time series,,
2020,London,-869461.0
2021,London,-989967.0


In [20]:
#Grösste Rückgänge in absoluten Zahlen pro Jahr (Flights)
loser_names = df_flights_unterschied.idxmin(axis=1)
loser_values = df_flights_unterschied.min(axis=1)
loser = pd.DataFrame(loser_names)
loser['Values'] = loser_values
loser

,0,Values
Time series,,
2020,London,-6026.0
2021,London,-6918.0


In [21]:
#Grösste Rückgänge in relativen Zahlen pro Jahr (Seats)
loser_names = df_seats_rel_unterschied.idxmin(axis=1)
loser_values = df_seats_rel_unterschied.min(axis=1)
loser = pd.DataFrame(loser_names)
loser['Values'] = loser_values
loser

,0,Values
Time series,,
2020,London,-0.511527
2021,London,-2.593174


In [22]:
#Grösste Rückgänge in relativen Zahlen pro Jahr (Flights)
loser_names = df_flights_rel_unterschied.idxmin(axis=1)
loser_values = df_flights_rel_unterschied.min(axis=1)
loser = pd.DataFrame(loser_names)
loser['Values'] = loser_values
loser

,0,Values
Time series,,
2020,Lugano,-0.910873
2021,London,-3.305355


In [23]:
#Die drei grössten Verlierer pro Jahr (Seats)
minima = []
for column in df_seats_unterschied.T:
    min3 = df_seats_unterschied.T.nsmallest(3, columns=column)
    minima.append(min3[column])
loser3 = pd.DataFrame(minima)
loser3

Ort,London,Berlin,Wien,Düsseldorf
2020,-869461.0,-465824.0,-446840.0,NaN
2021,-989967.0,-540443.0,NaN,-473206.0


In [24]:
#Die drei grössten Verlierer pro Jahr (Flüge)
minima = []
for column in df_flights_unterschied.T:
    min3 = df_flights_unterschied.T.nsmallest(3, columns=column)
    minima.append(min3[column])
loser3 = pd.DataFrame(minima)
loser3

Ort,London,Berlin,Düsseldorf
2020,-6026.0,-3119.0,-2660.0
2021,-6918.0,-3653.0,-3031.0


In [25]:
#Die drei grössten Verlierer pro Jahr (Seats relativ)
minima = []
for column in df_seats_rel_unterschied.T:
    min3 = df_seats_rel_unterschied.T.nsmallest(3, columns=column)
    minima.append(min3[column])
loser3 = pd.DataFrame(minima)
loser3

Ort,London,Barcelona,Paris,Düsseldorf,Berlin
2020,-0.511527,-0.485982,-0.451343,NaN,NaN
2021,-2.593174,NaN,NaN,-1.414191,-1.089609


In [26]:
#Die drei grössten Verlierer pro Jahr (Flights relativ)
minima = []
for column in df_flights_rel_unterschied.T:
    min3 = df_flights_rel_unterschied.T.nsmallest(3, columns=column)
    minima.append(min3[column])
loser3 = pd.DataFrame(minima)
loser3

Ort,Lugano,London,Stuttgart,Berlin,Düsseldorf
2020,-0.910873,-0.892493,-0.71844,NaN,NaN
2021,NaN,-3.305355,NaN,-1.456843,-1.438412


In [27]:
#Grösste Gewinne in absoluten Zahlen pro Jahr (Seats)
winner_names = df_seats_unterschied.idxmax(axis=1)
winner_values = df_seats_unterschied.max(axis=1)
winner = pd.DataFrame(winner_names)
winner['Values'] = winner_values
winner

,0,Values
Time series,,
2020,Dar Es Salaam,2715.0
2021,Thessaloniki,19421.0


In [28]:
#Grösste Gewinne in absoluten Zahlen pro Jahr (Flights)
winner_names = df_flights_unterschied.idxmax(axis=1)
winner_values = df_flights_unterschied.max(axis=1)
winner = pd.DataFrame(winner_names)
winner['Values'] = winner_values
winner

,0,Values
Time series,,
2020,Linz,68.0
2021,Thessaloniki,127.0


In [29]:
#Grösste Gewinne in relativen Zahlen pro Jahr (Seats)
winner_names = df_seats_rel_unterschied.idxmax(axis=1)
winner_values = df_seats_rel_unterschied.max(axis=1)
winner = pd.DataFrame(winner_names)
winner['Values'] = winner_values
winner

,0,Values
Time series,,
2020,Doha,0.738585
2021,Palma De Mallorca,1.529652


In [30]:
#Konkrete Angaben zu Palma
df_seats_rel.filter(like="Palma De Mallorca")

Ort,Palma De Mallorca
Time series,
2017,1.763464
2018,1.458457
2019,1.415108
2020,1.157224
2021,2.944759


In [31]:
#Grösste Gewinne in relativen Zahlen pro Jahr (Flights)
winner_names = df_flights_rel_unterschied.idxmax(axis=1)
winner_values = df_flights_rel_unterschied.max(axis=1)
winner = pd.DataFrame(winner_names)
winner['Values'] = winner_values
winner

,0,Values
Time series,,
2020,Frankfurt,0.958578
2021,Palma De Mallorca,1.501459


In [32]:
#Die drei grössten Gewinner pro Jahr (Seats)
maxima = []
for column in df_seats_unterschied.T:
    max3 = df_seats_unterschied.T.nlargest(3, columns=column)
    maxima.append(max3[column])
winner3 = pd.DataFrame(maxima)
winner3

Ort,Dar Es Salaam,Osaka,Menorca,Thessaloniki,Irakleion,Ibiza
2020,2715.0,2453.0,2436.0,NaN,NaN,NaN
2021,NaN,NaN,NaN,19421.0,17352.0,13470.0


In [33]:
#Die drei grössten Gewinner pro Jahr (Flights)
maxima = []
for column in df_flights_unterschied.T:
    max3 = df_flights_unterschied.T.nlargest(3, columns=column)
    maxima.append(max3[column])
winner3 = pd.DataFrame(maxima)
winner3

Ort,Linz,Menorca,Guernsey,Thessaloniki,Irakleion
2020,68.0,14.0,13.0,NaN,NaN
2021,86.0,NaN,NaN,127.0,69.0


In [34]:
#Die drei grössten Gewinner pro Jahr (Seats, relativ)
maxima = []
for column in df_seats_rel_unterschied.T:
    max3 = df_seats_rel_unterschied.T.nlargest(3, columns=column)
    maxima.append(max3[column])
winner3 = pd.DataFrame(maxima)
winner3

Ort,Doha,Porto,Frankfurt,Palma De Mallorca,Istanbul
2020,0.738585,0.716671,0.689236,NaN,NaN
2021,NaN,0.920906,NaN,1.529652,1.467733


In [35]:
#Die drei grössten Gewinner pro Jahr (Flights, relativ)
maxima = []
for column in df_flights_rel_unterschied.T:
    max3 = df_flights_rel_unterschied.T.nlargest(3, columns=column)
    maxima.append(max3[column])
winner3 = pd.DataFrame(maxima)
winner3

Ort,Frankfurt,Amsterdam,Belgrad,Palma De Mallorca,Istanbul
2020,0.958578,0.872116,0.674658,NaN,NaN
2021,NaN,NaN,0.908666,1.501459,1.389753
